In [72]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('tests-by-zcta.csv')
df_age = pd.read_csv('by-age.csv')
df_hosp = pd.read_csv('case-hosp-death.csv')
df_hosp['DATE_OF_INTEREST'] = pd.to_datetime(df_hosp['DATE_OF_INTEREST'])
df = df[1:]
df.nlargest(20, 'zcta_cum.perc_pos')

,MODZCTA,Positive,Total,zcta_cum.perc_pos
141,11368.0,1446,1861,77.70
142,11369.0,547,728,75.14
152,11411.0,271,368,73.64
144,11372.0,806,1102,73.14
82,11429.0,322,461,69.85
108,11219.0,1136,1644,69.10
145,11373.0,983,1425,68.98
127,11239.0,157,228,68.86
125,11237.0,274,408,67.16
168,11434.0,635,952,66.70


In [73]:
hosp = df_hosp
hosp = pd.melt(hosp, id_vars=['DATE_OF_INTEREST'], value_vars=['NEW_COVID_CASE_COUNT', 'HOSPITALIZED_CASE_COUNT', 'DEATH_COUNT'])
fig = px.line(hosp, x="DATE_OF_INTEREST", y='value', color='variable', title="Hospital Data - Raw Numbers")
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="7D",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)
fig.show()

In [74]:
cum_hosp_df = df_hosp
cum_hosp_df['NEW_COVID_CASE_COUNT'] = cum_hosp_df['NEW_COVID_CASE_COUNT'].cumsum()
cum_hosp_df['HOSPITALIZED_CASE_COUNT'] = cum_hosp_df['HOSPITALIZED_CASE_COUNT'].cumsum()
cum_hosp_df['DEATH_COUNT'] = cum_hosp_df['DEATH_COUNT'].cumsum()

cum_hosp_df = pd.melt(cum_hosp_df, id_vars=['DATE_OF_INTEREST'], value_vars=['NEW_COVID_CASE_COUNT', 'HOSPITALIZED_CASE_COUNT', 'DEATH_COUNT'])
fig = px.line(cum_hosp_df, x="DATE_OF_INTEREST", y='value', color='variable', title="Hospital Data - Cumulative Numbers")
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="7D",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)
fig.show()

In [75]:
fig = px.line(cum_hosp_df, x="DATE_OF_INTEREST", y='value', color='variable', log_y=True, title="Hospital Data - Cumulative Log Scale")
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="7D",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)
fig.show()

In [76]:
df_age_no_totals = df_age[:-1]
labels = df_age_no_totals['AGE_GROUP']

fig = make_subplots(cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['COVID Case Rate', 'Hospitalized Case Rate','Death Rate'])
fig.add_trace(go.Pie(labels=labels, values=df_age_no_totals['COVID_CASE_RATE'], scalegroup='one',
                     name="COVID Case Rate",sort=False), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=df_age_no_totals['HOSPITALIZED_CASE_RATE'], scalegroup='one',
                     name="Hospitalized Case Rate",sort=False), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=df_age_no_totals['DEATH_RATE'], scalegroup='one',
                     name="Death Rate",sort=False), 1, 3)

fig.update_layout(title_text='NYC COVID Rates per 100,000 people')
fig.show()

In [77]:
labels = df_age['AGE_GROUP']

fig = go.Figure(data=[
    go.Bar(name='COVID Case Rate', x=labels, y=df_age['COVID_CASE_RATE']),
    go.Bar(name='Hospitalized Case Rate', x=labels, y=df_age['HOSPITALIZED_CASE_RATE']),
    go.Bar(name='Death Rate', x=labels, y=df_age['DEATH_RATE'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title_text='NYC COVID Rates per 100,000 people')
fig.show()